<a href="https://colab.research.google.com/github/curtiscu/LYIT/blob/master/GatheringBaseMetrics_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gathering Metrics

# Setup env


In [0]:
# print all cell output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


## Google drive access

In [2]:
# mount google drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [3]:
# test, peek at data
! ls -al '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/'

# test, modules from local  'E:\Google Drive\LYIT\Dissertation\modules'
! ls -al '/content/drive/My Drive/LYIT/Dissertation/modules/'

total 35
-rw------- 1 root root 2589 Apr 27 12:01 10_soul-groove10_102_beat_4-4.mid
-rw------- 1 root root 4793 Apr 27 12:01 1_funk-groove1_138_beat_4-4.mid
-rw------- 1 root root 3243 Apr 27 12:01 2_funk-groove2_105_beat_4-4.mid
-rw------- 1 root root 4466 Apr 27 12:01 3_soul-groove3_86_beat_4-4.mid
-rw------- 1 root root 2551 Apr 27 12:01 4_soul-groove4_80_beat_4-4.mid
-rw------- 1 root root 3798 Apr 27 12:01 5_funk-groove5_84_beat_4-4.mid
-rw------- 1 root root 3760 Apr 27 12:01 6_hiphop-groove6_87_beat_4-4.mid
-rw------- 1 root root 1894 Apr 27 12:01 7_pop-groove7_138_beat_4-4.mid
-rw------- 1 root root 2437 Apr 27 12:01 8_rock-groove8_65_beat_4-4.mid
-rw------- 1 root root 3448 Apr 27 12:01 9_soul-groove9_105_beat_4-4.mid
total 34
-rw------- 1 root root 21727 Jun  4 19:53 data_prep.py
drwx------ 2 root root  4096 May 10 13:31 __pycache__
-rw------- 1 root root  8416 Jun  2 21:24 visualizations.py


## Auto reload module

Now using library code I've created and saved to google drive which is automatically pushed to the cloud and made available to the colab env. The autoreload stuff below should help imports to 'reimport' to load changes to the library code.

It's not the quickest/ most reliable, so if in a hurry, brute force loading of changes by restarting the runtime.

In [0]:
# tool to auto reload modules.
%load_ext autoreload

# config to auto-reload all modules, handy to make 
# writing and testing modules much easier.
%autoreload 2

## Imports and accessing lib functions

In [5]:
# install required libs
!pip install mido

In [6]:
# import my modules
import sys
sys.path.append('/content/drive/My Drive/LYIT/Dissertation/modules/')
import data_prep as dp

LOADING custom module, when: 2020-06-11 23:27:03.053190, module name: data_prep


In [0]:
# imports
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np


# object that provides colours for charts
from itertools import cycle



In [8]:
# testing auto reload of modules 
dp.test_function_call('bling')

Test function called worked! when: 2020-06-11 23:27:03.089910,  param:bling


## Pandas display options

In [0]:
def set_pandas_display_options() -> None:
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = 1000
    display.max_rows = 2000
    display.max_colwidth = 1000
    display.width = None
    # display.precision = 2  # set as needed

set_pandas_display_options()
#pd.reset_option('all')


# Load some data

In [10]:

file_1 = '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid'
file_7 = '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer7/eval_session/1_funk-groove1_138_beat_4-4.mid'

file_df, f, mtt = dp.load_file(file_1)  # load data

FILE name: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid
    loaded file: <midi file '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid' type 0, 1 tracks, 1300 messages>
    time sig: <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
    tempo: <meta message set_tempo tempo=434783 time=0>
    track count: 1, tracks: [<midi track 'MIDI' 1300 messages>]
    MIDI file type: 0
    > processing track: <midi track 'MIDI' 1300 messages>
    note_on span - first tick: 3 , last tick: 30658 
    good instruments: 6, {36.0: 'Bass Drum 1 (36)', 38.0: 'Acoustic Snare (38)', 42.0: 'Closed Hi Hat (42)', 43.0: 'High Floor Tom (43)', 49.0: 'Crash Cymbal 1 (49)', 51.0: 'Ride Cymbal 1 (51)'}
    bar info - bars in file: 16, bar quantize level: 16.0
    tick info - ticks per time sig beat: 480.0, ticks per quantize bin: 120


In [11]:
file_df.head()
file_df.info()

track_msg_num  total_ticks  note  velocity  \
bar_number bar_beat_number note                                              
1          1               49             0:6            3    49        60   
                           51             0:7            7    51        55   
                           36             0:8           10    36        64   
           3               36            0:14          211    36        57   
                           51            0:16          238    51        31   

                                 beat_offset beat_center  bar_number  \
bar_number bar_beat_number note                                        
1          1               49              3           0           1   
                           51              7           0           1   
                           36             10           0           1   
           3               36            -29         240           1   
                           51             -2         240           1   

                                 bar_beat_number  
bar_number bar_beat_number note                   
1          1               49                  1  
                           51                  1  
                           36                  1  
           3               36                  3  
                           51                  3

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 412 entries, (1, 1, 49) to (16, 16, 42)
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   track_msg_num    412 non-null    string  
 1   total_ticks      412 non-null    int64   
 2   note             412 non-null    int64   
 3   velocity         412 non-null    int64   
 4   beat_offset      412 non-null    int64   
 5   beat_center      412 non-null    category
 6   bar_number       412 non-null    int64   
 7   bar_beat_number  412 non-null    int64   
dtypes: category(1), int64(6), string(1)
memory usage: 25.2 KB


In [0]:
tmp_df = file_df.copy()


# GroupBy tests

Very good info on using DataFrame groupby operations...
* https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html
* https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/
* https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html
* https://chrisalbon.com/python/data_wrangling/pandas_apply_operations_to_groups/
* https://stackoverflow.com/a/51231000
* http://www.gregreda.com/2013/10/26/working-with-pandas-dataframes/



In [0]:
# calculate number of hits per metric position 
op = tmp_df.groupby(level=1)['note'].count()  

In [14]:
print(str(op))

bar_beat_number
1     54
2      7
3     43
4      7
5     48
6      2
7     30
8     16
9     34
10    20
11    44
12     8
13    40
14     2
15    37
16    20
Name: note, dtype: int64


We have duplicate columns & index names, need to drop some stuff to explore more.

In [0]:
tmp2 = tmp_df.drop(columns=['bar_beat_number', 'note'])

In [16]:
tmp2.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 412 entries, (1, 1, 49) to (16, 16, 42)
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   track_msg_num  412 non-null    string  
 1   total_ticks    412 non-null    int64   
 2   velocity       412 non-null    int64   
 3   beat_offset    412 non-null    int64   
 4   beat_center    412 non-null    category
 5   bar_number     412 non-null    int64   
dtypes: category(1), int64(4), string(1)
memory usage: 18.7 KB


In [0]:
# calculate number of hits per instrument, per metric position 
op2 = tmp2.groupby(by=['bar_beat_number', 'note'])['velocity'].count()

In [18]:
op2

bar_beat_number  note
1                36      12
                 38      10
                 42      15
                 49       1
                 51      16
2                38       5
                 42       2
3                36      16
                 42      11
                 51      16
4                36       4
                 51       3
5                38      16
                 42      16
                 51      16
6                42       2
7                42      14
                 51      16
8                38      16
9                42      17
                 51      17
10               38      17
                 42       3
11               36      15
                 42      13
                 43       1
                 49       3
                 51      12
12               36       6
                 38       2
13               38       8
                 42      16
                 51      16
14               36       1
                 42       

In [0]:
op3 = pd.DataFrame(op2)

In [0]:
op3.columns = ['hit_count']

# Frequency table

Handy to know..
* https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html

In [21]:
op3

hit_count
bar_beat_number note           
1               36           12
                38           10
                42           15
                49            1
                51           16
2               38            5
                42            2
3               36           16
                42           11
                51           16
4               36            4
                51            3
5               38           16
                42           16
                51           16
6               42            2
7               42           14
                51           16
8               38           16
9               42           17
                51           17
10              38           17
                42            3
11              36           15
                42           13
                43            1
                49            3
                51           12
12              36            6
                38            2
13              38            8
                42           16
                51           16
14              36            1
                42            1
15              38            8
                42           13
                51           16
16              38           19
                42            1

In [0]:
# flip things around, fill with '0' instead of NaN, then convert to int
op4 = op3.unstack(0).fillna(0).astype('int32')

In [23]:
op4

hit_count                                                     \
bar_beat_number        1  2   3  4   5  6   7   8   9   10  11 12  13 14  15   
note                                                                           
36                     12  0  16  4   0  0   0   0   0   0  15  6   0  1   0   
38                     10  5   0  0  16  0   0  16   0  17   0  2   8  0   8   
42                     15  2  11  0  16  2  14   0  17   3  13  0  16  1  13   
43                      0  0   0  0   0  0   0   0   0   0   1  0   0  0   0   
49                      1  0   0  0   0  0   0   0   0   0   3  0   0  0   0   
51                     16  0  16  3  16  0  16   0  17   0  12  0  16  0  16   

                     
bar_beat_number  16  
note                 
36                0  
38               19  
42                1  
43                0  
49                0  
51                0

## Flesh out missing quantize buckets
Some metric positions don't have any beats, for completeness/ presentation, do a bit of extra work to flesh out those columns and fill with zeros.
* https://stackoverflow.com/a/56907742

In [0]:
op4.columns = op4.columns.droplevel(level=0)    

In [25]:
fill_cols = pd.DataFrame(columns=[x for x in range(1, 17)])
fill_cols

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16


In [26]:
full_col_list = list(set().union(op4.columns, fill_cols.columns))
full_col_list


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

In [0]:
op5 = op4.reindex(columns=full_col_list, fill_value=0)

In [28]:
op5

bar_beat_number,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
note,,,,,,,,,,,,,,,,
36,12,0,16,4,0,0,0,0,0,0,15,6,0,1,0,0
38,10,5,0,0,16,0,0,16,0,17,0,2,8,0,8,19
42,15,2,11,0,16,2,14,0,17,3,13,0,16,1,13,1
43,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
49,1,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0
51,16,0,16,3,16,0,16,0,17,0,12,0,16,0,16,0


In [0]:
from data_prep import MidiTools as mt

In [30]:
mt.getInstruments(op5.index.values)

['Bass Drum 1 (36)',
 'Acoustic Snare (38)',
 'Closed Hi Hat (42)',
 'High Floor Tom (43)',
 'Crash Cymbal 1 (49)',
 'Ride Cymbal 1 (51)']

In [31]:
new_row_index = dict(zip(op5.index.values, mt.getInstruments(op5.index.values)))
new_row_index

{36: 'Bass Drum 1 (36)',
 38: 'Acoustic Snare (38)',
 42: 'Closed Hi Hat (42)',
 43: 'High Floor Tom (43)',
 49: 'Crash Cymbal 1 (49)',
 51: 'Ride Cymbal 1 (51)'}

In [0]:
op6 = op5.rename(new_row_index)

## Complete Frequency Table
  

In [33]:
op6

bar_beat_number,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
note,,,,,,,,,,,,,,,,
Bass Drum 1 (36),12,0,16,4,0,0,0,0,0,0,15,6,0,1,0,0
Acoustic Snare (38),10,5,0,0,16,0,0,16,0,17,0,2,8,0,8,19
Closed Hi Hat (42),15,2,11,0,16,2,14,0,17,3,13,0,16,1,13,1
High Floor Tom (43),0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
Crash Cymbal 1 (49),1,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0
Ride Cymbal 1 (51),16,0,16,3,16,0,16,0,17,0,12,0,16,0,16,0


# Detect multiple hits in quantize bucket

This detects multiple hits of the same instrument, within our quantize window - this is a problem. Need to decide what to do here, maybe ignore all these hits by pre-filtering?

In [0]:
m_hits = file_df.copy()

In [0]:
m_hits.drop(columns=['note'], inplace=True)

In [36]:
m_hits.groupby(['beat_center','note']).count().head(10)

track_msg_num  total_ticks  velocity  beat_offset  \
beat_center note                                                      
0           36              1.0          1.0       1.0          1.0   
            38              NaN          NaN       NaN          NaN   
            42              NaN          NaN       NaN          NaN   
            43              NaN          NaN       NaN          NaN   
            49              1.0          1.0       1.0          1.0   
            51              1.0          1.0       1.0          1.0   
120         36              NaN          NaN       NaN          NaN   
            38              NaN          NaN       NaN          NaN   
            42              NaN          NaN       NaN          NaN   
            43              NaN          NaN       NaN          NaN   

                  bar_number  bar_beat_number  
beat_center note                               
0           36           1.0              1.0  
            38           NaN              NaN  
            42           NaN              NaN  
            43           NaN              NaN  
            49           1.0              1.0  
            51           1.0              1.0  
120         36           NaN              NaN  
            38           NaN              NaN  
            42           NaN              NaN  
            43           NaN              NaN

Handy filter functions..
* https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#filtration

Lets us filter entire dataframe groupby result to only show groups with more than 1 member, i.e. duplicate hits in bucket.

**Basically the output below must (currently) be flagged as an error**

In [37]:
# filters entire dataframe to only show groups
# with more than 1 member, i.e. duplicate htis in bucket
m_hits.groupby(['beat_center','note']).filter(lambda a_group: len(a_group) > 1)

track_msg_num  total_ticks  velocity  \
bar_number bar_beat_number note                                        
1          9               42            0:41          908        77   
                           51            0:43          940        46   
                           42            0:45          971         0   
                           51            0:46          971         0   
2          16              38           0:155         3710        37   
                           38           0:158         3747        41   
3          1               38           0:161         3796        33   
                           38           0:168         3846        23   
4          10              38           0:289         6813        37   
                           38           0:290         6867        20   
5          16              38           0:401         9468        37   
                           38           0:404         9506        33   
                           38           0:406         9532        30   
6          16              38           0:487        11397        36   
                           38           0:491        11441        32   
9          16              38           0:730        17147        38   
                           38           0:733        17182        38   
10         16              38           0:814        19056        43   
                           38           0:817        19092        46   
                           38           0:820        19116        40   
13         16              38          0:1056        24797        33   
                           38          0:1059        24845        32   
                           38          0:1062        24876        32   
14         16              38          0:1140        26733        40   
                           38          0:1143        26778        41   
                           38          0:1145        26805        34   

                                 beat_offset beat_center  bar_number  \
bar_number bar_beat_number note                                        
1          9               42            -52         960           1   
                           51            -20         960           1   
                           42             11         960           1   
                           51             11         960           1   
2          16              38            -10        3720           2   
                           38             27        3720           2   
3          1               38            -44        3840           3   
                           38              6        3840           3   
4          10              38            -27        6840           4   
                           38             27        6840           4   
5          16              38            -12        9480           5   
                           38             26        9480           5   
                           38             52        9480           5   
6          16              38             -3       11400           6   
                           38             41       11400           6   
9          16              38            -13       17160           9   
                           38             22       17160           9   
10         16              38            -24       19080          10   
                           38             12       19080          10   
                           38             36       19080          10   
13         16              38            -43       24840          13   
                           38              5       24840          13   
                           38             36       24840          13   
14         16              38            -27       26760          14   
                           38             18       26760          14   
                           38             45       26760          14   

# Groupby over x number bars

*DON'T KNOW IF I NEED THIS*

Was considering spreading out the histogram table over 2 bars (32 beats).

In [0]:
bins_32 = file_df.copy()
#bins_32.head()

# Gather other stats (mean, median, min/ max, std, etc.)

Handy reference..
* https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/


In [0]:
stats_df = file_df.drop(columns=['bar_beat_number', 'note', 'track_msg_num', 'total_ticks', 'beat_center', 'bar_number' ]).copy()

In [40]:
stats_df.head()

velocity  beat_offset
bar_number bar_beat_number note                       
1          1               49          60            3
                           51          55            7
                           36          64           10
           3               36          57          -29
                           51          31           -2

## Build stats

In [0]:
stats_1 = stats_df.groupby(by=['bar_beat_number', 'note']).agg(
    hits=('velocity', "count"),     # count of hits at position
    offset_mean=('beat_offset', np.mean),   # offset mean
    offset_std=('beat_offset', np.std),    # offset standard deviation
    velocity_mean=('velocity', np.mean),      # velocity mean
    velocity_std=('velocity', np.std)        # velocity standard deviation
    )

In [42]:
stats_1.head()

hits  offset_mean  offset_std  velocity_mean  \
bar_beat_number note                                                 
1               36      12    -9.250000   12.144022      60.500000   
                38      10   -20.800000   42.202686      28.800000   
                42      15   -28.933333   14.129335      77.066667   
                49       1     3.000000         NaN      60.000000   
                51      16    -2.937500   10.828781      50.500000   

                      velocity_std  
bar_beat_number note                
1               36        5.452272  
                38        4.491968  
                42        3.712271  
                49             NaN  
                51        6.491019

Smooth out our decimal places...
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.round.html


In [43]:
stats_1 = stats_1.round(2)
stats_1

hits  offset_mean  offset_std  velocity_mean  \
bar_beat_number note                                                 
1               36      12        -9.25       12.14          60.50   
                38      10       -20.80       42.20          28.80   
                42      15       -28.93       14.13          77.07   
                49       1         3.00         NaN          60.00   
                51      16        -2.94       10.83          50.50   
2               38       5       -21.40       14.84          35.40   
                42       2        51.50        9.19          45.00   
3               36      16       -10.56       10.75          61.50   
                42      11       -37.36        9.87          45.18   
                51      16       -11.06       12.40          44.31   
4               36       4        -8.00       12.36          53.50   
                51       3        -8.67        3.06          44.33   
5               38      16        -4.94       10.04         100.06   
                42      16       -24.38       14.01          81.25   
                51      16        -3.69       12.21          57.81   
6               42       2        54.50        0.71          53.50   
7               42      14       -35.57       10.30          47.71   
                51      16         1.38       12.88          42.25   
8               38      16       -11.69       14.99          88.44   
9               42      17       -24.71       15.72          72.18   
                51      17        -2.00       14.54          49.41   
10              38      17        -4.94       16.77          30.65   
                42       3        53.67        2.52          37.00   
11              36      15       -25.87       18.63          64.53   
                42      13       -39.46       11.51          42.31   
                43       1       -21.00         NaN         127.00   
                49       3       -28.67       11.15          66.67   
                51      12        -0.08       13.26          45.83   
12              36       6       -17.33       13.35          56.50   
                38       2        31.00       33.94          21.00   
13              38       8       -11.25       10.73         102.00   
                42      16       -30.31       12.87          80.12   
                51      16        -9.88       12.27          55.19   
14              36       1         6.00         NaN          52.00   
                42       1        57.00         NaN          42.00   
15              38       8       -15.75       17.49         109.00   
                42      13       -41.54       11.45          46.00   
                51      16        -7.38       14.75          54.81   
16              38      19         9.74       27.05          36.47   
                42       1        58.00         NaN          25.00   

                      velocity_std  
bar_beat_number note                
1               36            5.45  
                38            4.49  
                42            3.71  
                49             NaN  
                51            6.49  
2               38            4.10  
                42           14.14  
3               36           18.30  
                42           13.72  
                51            5.40  
4               36            3.87  
                51            5.03  
5               38            6.48  
                42            2.89  
                51            7.00  
6               42           12.02  
7               42           12.96  
                51            3.84  
8               38           23.66  
9               42           18.82  
                51           15.25  
10              38            7.03  
                42           13.23  
11              36           21.73  
                42           13.33  
                43             NaN  
                49           15.31  
        

In [44]:
instrument_list = stats_1.index.unique(level=1)._data
instrument_map = dict(zip(instrument_list, mt.getInstruments(instrument_list)))
instrument_map

{36: 'Bass Drum 1 (36)',
 38: 'Acoustic Snare (38)',
 42: 'Closed Hi Hat (42)',
 43: 'High Floor Tom (43)',
 49: 'Crash Cymbal 1 (49)',
 51: 'Ride Cymbal 1 (51)'}

## Completed stats matrix (v1)
This one is a bit long winded, sideways speaking ...

In [45]:
stats_1.T.stack(1).unstack(0).fillna(0)

bar_beat_number    1                                                      2   \
                 hits offset_mean offset_std velocity_mean velocity_std hits   
note                                                                           
36               12.0       -9.25      12.14         60.50         5.45  0.0   
38               10.0      -20.80      42.20         28.80         4.49  5.0   
42               15.0      -28.93      14.13         77.07         3.71  2.0   
43                0.0        0.00       0.00          0.00         0.00  0.0   
49                1.0        3.00       0.00         60.00         0.00  0.0   
51               16.0       -2.94      10.83         50.50         6.49  0.0   

bar_beat_number                                                      3   \
                offset_mean offset_std velocity_mean velocity_std  hits   
note                                                                      
36                      0.0       0.00           0.0         0.00  16.0   
38                    -21.4      14.84          35.4         4.10   0.0   
42                     51.5       9.19          45.0        14.14  11.0   
43                      0.0       0.00           0.0         0.00   0.0   
49                      0.0       0.00           0.0         0.00   0.0   
51                      0.0       0.00           0.0         0.00  16.0   

bar_beat_number                                                     4   \
                offset_mean offset_std velocity_mean velocity_std hits   
note                                                                     
36                   -10.56      10.75         61.50        18.30  4.0   
38                     0.00       0.00          0.00         0.00  0.0   
42                   -37.36       9.87         45.18        13.72  0.0   
43                     0.00       0.00          0.00         0.00  0.0   
49                     0.00       0.00          0.00         0.00  0.0   
51                   -11.06      12.40         44.31         5.40  3.0   

bar_beat_number                                                      5   \
                offset_mean offset_std velocity_mean velocity_std  hits   
note                                                                      
36                    -8.00      12.36         53.50         3.87   0.0   
38                     0.00       0.00          0.00         0.00  16.0   
42                     0.00       0.00          0.00         0.00  16.0   
43                     0.00       0.00          0.00         0.00   0.0   
49                     0.00       0.00          0.00         0.00   0.0   
51                    -8.67       3.06         44.33         5.03  16.0   

bar_beat_number                                                     6   \
                offset_mean offset_std velocity_mean velocity_std hits   
note                                                                     
36                     0.00       0.00          0.00         0.00  0.0   
38                    -4.94      10.04        100.06         6.48  0.0   
42                   -24.38      14.01         81.25         2.89  2.0   
43                     0.00       0.00          0.00         0.00  0.0   
49                     0.00       0.00          0.00         0.00  0.0   
51                    -3.69      12.21         57.81         7.00  0.0   

bar_beat_number                                                      7   \
                offset_mean offset_std velocity_mean velocity_std  hits   
note                                                                      
36                      0.0       0.00           0.0         0.00   0.0   
38                      0.0       0.00           0.0         0.00   0.0   
42                     54.5       0.71          53.5        12.02  14.0   
43                      0.0       0.00           0.0         0.00   0.0   
49                      0.0       0.00           0.0         0.00   

## Completed stats matrix (v2)
Little easier on the eyes...

In [46]:
stats_2 = stats_1.T.stack(1).fillna(0) #.unstack(0).fillna(0)
stats_2

bar_beat_number        1      2      3      4       5      6      7      8   \
              note                                                            
hits          36    12.00   0.00  16.00   4.00    0.00   0.00   0.00   0.00   
              38    10.00   5.00   0.00   0.00   16.00   0.00   0.00  16.00   
              42    15.00   2.00  11.00   0.00   16.00   2.00  14.00   0.00   
              43     0.00   0.00   0.00   0.00    0.00   0.00   0.00   0.00   
              49     1.00   0.00   0.00   0.00    0.00   0.00   0.00   0.00   
              51    16.00   0.00  16.00   3.00   16.00   0.00  16.00   0.00   
offset_mean   36    -9.25   0.00 -10.56  -8.00    0.00   0.00   0.00   0.00   
              38   -20.80 -21.40   0.00   0.00   -4.94   0.00   0.00 -11.69   
              42   -28.93  51.50 -37.36   0.00  -24.38  54.50 -35.57   0.00   
              43     0.00   0.00   0.00   0.00    0.00   0.00   0.00   0.00   
              49     3.00   0.00   0.00   0.00    0.00   0.00   0.00   0.00   
              51    -2.94   0.00 -11.06  -8.67   -3.69   0.00   1.38   0.00   
offset_std    36    12.14   0.00  10.75  12.36    0.00   0.00   0.00   0.00   
              38    42.20  14.84   0.00   0.00   10.04   0.00   0.00  14.99   
              42    14.13   9.19   9.87   0.00   14.01   0.71  10.30   0.00   
              49     0.00   0.00   0.00   0.00    0.00   0.00   0.00   0.00   
              51    10.83   0.00  12.40   3.06   12.21   0.00  12.88   0.00   
velocity_mean 36    60.50   0.00  61.50  53.50    0.00   0.00   0.00   0.00   
              38    28.80  35.40   0.00   0.00  100.06   0.00   0.00  88.44   
              42    77.07  45.00  45.18   0.00   81.25  53.50  47.71   0.00   
              43     0.00   0.00   0.00   0.00    0.00   0.00   0.00   0.00   
              49    60.00   0.00   0.00   0.00    0.00   0.00   0.00   0.00   
              51    50.50   0.00  44.31  44.33   57.81   0.00  42.25   0.00   
velocity_std  36     5.45   0.00  18.30   3.87    0.00   0.00   0.00   0.00   
              38     4.49   4.10   0.00   0.00    6.48   0.00   0.00  23.66   
              42     3.71  14.14  13.72   0.00    2.89  12.02  12.96   0.00   
              49     0.00   0.00   0.00   0.00    0.00   0.00   0.00   0.00   
              51     6.49   0.00   5.40   5.03    7.00   0.00   3.84   0.00   

bar_beat_number        9      10      11     12      13    14      15     16  
              note                                                            
hits          36     0.00   0.00   15.00   6.00    0.00   1.0    0.00   0.00  
              38     0.00  17.00    0.00   2.00    8.00   0.0    8.00  19.00  
              42    17.00   3.00   13.00   0.00   16.00   1.0   13.00   1.00  
              43     0.00   0.00    1.00   0.00    0.00   0.0    0.00   0.00  
              49     0.00   0.00    3.00   0.00    0.00   0.0    0.00   0.00  
              51    17.00   0.00   12.00   0.00   16.00   0.0   16.00   0.00  
offset_mean   36     0.00   0.00  -25.87 -17.33    0.00   6.0    0.00   0.00  
              38     0.00  -4.94    0.00  31.00  -11.25   0.0  -15.75   9.74  
              42   -24.71  53.67  -39.46   0.00  -30.31  57.0  -41.54  58.00  
              43     0.00   0.00  -21.00   0.00    0.00   0.0    0.00   0.00  
              49     0.00   0.00  -28.67   0.00    0.00   0.0    0.00   0.00  
              51    -2.00   0.00   -0.08   0.00   -9.88   0.0   -7.38   0.00  
offset_std    36     0.00   0.00   18.63  13.35    0.00   0.0    0.00   0.00  
              38     0.00  16.77    0.00  33.94   10.73   0.0   17.49  27.05  
              42    15.72   2.52   11.51   0.00   12.87   0.0   11.45   0.00  
              49     0.00   0.00   11.15   0.00    0.00   0.0    0.00   0.00  
              51    14.54   0.00   13.26   0.00   12.27   0.0   14.75   0.00  
velocity_mean 36     0.00   0.00   64.53  56.50    0.00  52.0    0.00   0.00  
              38     0.00  30.65    0.00  21.00  

Flesh out missing metric placeholders...

In [47]:
stats_2 = stats_2.reindex(columns=full_col_list, fill_value=0)
stats_2

bar_beat_number        1      2      3      4       5      6      7      8   \
              note                                                            
hits          36    12.00   0.00  16.00   4.00    0.00   0.00   0.00   0.00   
              38    10.00   5.00   0.00   0.00   16.00   0.00   0.00  16.00   
              42    15.00   2.00  11.00   0.00   16.00   2.00  14.00   0.00   
              43     0.00   0.00   0.00   0.00    0.00   0.00   0.00   0.00   
              49     1.00   0.00   0.00   0.00    0.00   0.00   0.00   0.00   
              51    16.00   0.00  16.00   3.00   16.00   0.00  16.00   0.00   
offset_mean   36    -9.25   0.00 -10.56  -8.00    0.00   0.00   0.00   0.00   
              38   -20.80 -21.40   0.00   0.00   -4.94   0.00   0.00 -11.69   
              42   -28.93  51.50 -37.36   0.00  -24.38  54.50 -35.57   0.00   
              43     0.00   0.00   0.00   0.00    0.00   0.00   0.00   0.00   
              49     3.00   0.00   0.00   0.00    0.00   0.00   0.00   0.00   
              51    -2.94   0.00 -11.06  -8.67   -3.69   0.00   1.38   0.00   
offset_std    36    12.14   0.00  10.75  12.36    0.00   0.00   0.00   0.00   
              38    42.20  14.84   0.00   0.00   10.04   0.00   0.00  14.99   
              42    14.13   9.19   9.87   0.00   14.01   0.71  10.30   0.00   
              49     0.00   0.00   0.00   0.00    0.00   0.00   0.00   0.00   
              51    10.83   0.00  12.40   3.06   12.21   0.00  12.88   0.00   
velocity_mean 36    60.50   0.00  61.50  53.50    0.00   0.00   0.00   0.00   
              38    28.80  35.40   0.00   0.00  100.06   0.00   0.00  88.44   
              42    77.07  45.00  45.18   0.00   81.25  53.50  47.71   0.00   
              43     0.00   0.00   0.00   0.00    0.00   0.00   0.00   0.00   
              49    60.00   0.00   0.00   0.00    0.00   0.00   0.00   0.00   
              51    50.50   0.00  44.31  44.33   57.81   0.00  42.25   0.00   
velocity_std  36     5.45   0.00  18.30   3.87    0.00   0.00   0.00   0.00   
              38     4.49   4.10   0.00   0.00    6.48   0.00   0.00  23.66   
              42     3.71  14.14  13.72   0.00    2.89  12.02  12.96   0.00   
              49     0.00   0.00   0.00   0.00    0.00   0.00   0.00   0.00   
              51     6.49   0.00   5.40   5.03    7.00   0.00   3.84   0.00   

bar_beat_number        9      10      11     12      13    14      15     16  
              note                                                            
hits          36     0.00   0.00   15.00   6.00    0.00   1.0    0.00   0.00  
              38     0.00  17.00    0.00   2.00    8.00   0.0    8.00  19.00  
              42    17.00   3.00   13.00   0.00   16.00   1.0   13.00   1.00  
              43     0.00   0.00    1.00   0.00    0.00   0.0    0.00   0.00  
              49     0.00   0.00    3.00   0.00    0.00   0.0    0.00   0.00  
              51    17.00   0.00   12.00   0.00   16.00   0.0   16.00   0.00  
offset_mean   36     0.00   0.00  -25.87 -17.33    0.00   6.0    0.00   0.00  
              38     0.00  -4.94    0.00  31.00  -11.25   0.0  -15.75   9.74  
              42   -24.71  53.67  -39.46   0.00  -30.31  57.0  -41.54  58.00  
              43     0.00   0.00  -21.00   0.00    0.00   0.0    0.00   0.00  
              49     0.00   0.00  -28.67   0.00    0.00   0.0    0.00   0.00  
              51    -2.00   0.00   -0.08   0.00   -9.88   0.0   -7.38   0.00  
offset_std    36     0.00   0.00   18.63  13.35    0.00   0.0    0.00   0.00  
              38     0.00  16.77    0.00  33.94   10.73   0.0   17.49  27.05  
              42    15.72   2.52   11.51   0.00   12.87   0.0   11.45   0.00  
              49     0.00   0.00   11.15   0.00    0.00   0.0    0.00   0.00  
              51    14.54   0.00   13.26   0.00   12.27   0.0   14.75   0.00  
velocity_mean 36     0.00   0.00   64.53  56.50    0.00  52.0    0.00   0.00  
              38     0.00  30.65    0.00  21.00  

## Completed stats matrix (v3)

In [0]:
keys=stats_2.index.unique(level=1)._data

In [49]:
keys

array([36, 38, 42, 43, 49, 51])

In [50]:

for i in keys:
  print('\n\n >>>> Showing stats for instrument: {}'.format(instrument_map[i]))
  #stats_2[stats_2.index.isin([i], level=1)]#.droplevel('note')
  stats_2[stats_2.index.isin([i], level=1)].droplevel('note')




 >>>> Showing stats for instrument: Bass Drum 1 (36)


bar_beat_number,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
hits,12.00,0.0,16.00,4.00,0.0,0.0,0.0,0.0,0.0,0.0,15.00,6.00,0.0,1.0,0.0,0.0
offset_mean,-9.25,0.0,-10.56,-8.00,0.0,0.0,0.0,0.0,0.0,0.0,-25.87,-17.33,0.0,6.0,0.0,0.0
offset_std,12.14,0.0,10.75,12.36,0.0,0.0,0.0,0.0,0.0,0.0,18.63,13.35,0.0,0.0,0.0,0.0
velocity_mean,60.50,0.0,61.50,53.50,0.0,0.0,0.0,0.0,0.0,0.0,64.53,56.50,0.0,52.0,0.0,0.0
velocity_std,5.45,0.0,18.30,3.87,0.0,0.0,0.0,0.0,0.0,0.0,21.73,6.02,0.0,0.0,0.0,0.0




 >>>> Showing stats for instrument: Acoustic Snare (38)


bar_beat_number,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
hits,10.00,5.00,0.0,0.0,16.00,0.0,0.0,16.00,0.0,17.00,0.0,2.00,8.00,0.0,8.00,19.00
offset_mean,-20.80,-21.40,0.0,0.0,-4.94,0.0,0.0,-11.69,0.0,-4.94,0.0,31.00,-11.25,0.0,-15.75,9.74
offset_std,42.20,14.84,0.0,0.0,10.04,0.0,0.0,14.99,0.0,16.77,0.0,33.94,10.73,0.0,17.49,27.05
velocity_mean,28.80,35.40,0.0,0.0,100.06,0.0,0.0,88.44,0.0,30.65,0.0,21.00,102.00,0.0,109.00,36.47
velocity_std,4.49,4.10,0.0,0.0,6.48,0.0,0.0,23.66,0.0,7.03,0.0,18.38,10.04,0.0,7.37,4.59




 >>>> Showing stats for instrument: Closed Hi Hat (42)


...

bar_beat_number,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
hits,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
offset_mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-21.0,0.0,0.0,0.0,0.0,0.0
velocity_mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,127.0,0.0,0.0,0.0,0.0,0.0




 >>>> Showing stats for instrument: Crash Cymbal 1 (49)


bar_beat_number,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
hits,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.00,0.0,0.0,0.0,0.0,0.0
offset_mean,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-28.67,0.0,0.0,0.0,0.0,0.0
offset_std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.15,0.0,0.0,0.0,0.0,0.0
velocity_mean,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,66.67,0.0,0.0,0.0,0.0,0.0
velocity_std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.31,0.0,0.0,0.0,0.0,0.0




 >>>> Showing stats for instrument: Ride Cymbal 1 (51)


bar_beat_number,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
hits,16.00,0.0,16.00,3.00,16.00,0.0,16.00,0.0,17.00,0.0,12.00,0.0,16.00,0.0,16.00,0.0
offset_mean,-2.94,0.0,-11.06,-8.67,-3.69,0.0,1.38,0.0,-2.00,0.0,-0.08,0.0,-9.88,0.0,-7.38,0.0
offset_std,10.83,0.0,12.40,3.06,12.21,0.0,12.88,0.0,14.54,0.0,13.26,0.0,12.27,0.0,14.75,0.0
velocity_mean,50.50,0.0,44.31,44.33,57.81,0.0,42.25,0.0,49.41,0.0,45.83,0.0,55.19,0.0,54.81,0.0
velocity_std,6.49,0.0,5.40,5.03,7.00,0.0,3.84,0.0,15.25,0.0,5.84,0.0,11.03,0.0,14.61,0.0
